In [1]:
!pip install schedule
!pip install snowflake-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Connecting to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

In [32]:
query=      """
            SELECT *
            FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
            """
cur = conn.cursor()
cur.execute(f"USE {database}.{schema}")

result_df = pd.read_sql_query(query, conn)
print("\n",result_df['DATE'])

C:\Users\User\AppData\Local\Temp\ipykernel_4940\2615130172.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)



 0         2022-10-27
1         2022-10-18
2         2022-10-21
3         2022-10-21
4         2022-10-21
             ...    
799800    2023-07-05
799801    2023-06-15
799802    2023-06-27
799803    2023-06-28
799804    2022-02-15
Name: DATE, Length: 799805, dtype: object


In [33]:
# Identify columns of object type
object_columns = result_df.select_dtypes(include='object').columns

In [28]:
object_columns

Index(['RETAILER_NAME', 'ONBOARDING_STATUS', 'DATE', 'PARENT_RETAILER_NAME',
       'AREA_CLASSIFICATION', 'CITY_NAME', 'BRAND_TYPE', 'SHORT_NAME', 'SKU',
       'PRODUCT_TYPE', 'PRODUCT_NAME', 'RETAILER_CATEGORY', 'SALE_TYPE',
       'DISTRIBUTOR'],
      dtype='object')

In [34]:
# Convert object columns to datetime
for col in object_columns:
    try:
        converted_values = pd.to_datetime(result_df[col], errors='coerce')
        if converted_values.notna().any():
            result_df[col] = converted_values
    except ValueError:
        # Handle any specific errors that might occur during conversion
        pass

In [35]:
result_df.head()

,RETAILER_NAME,ONBOARDING_STATUS,DATE,QUANTITY,SKU_ORDER_PRICE_AMOUNT,NET_SALE,PARENT_RETAILER_NAME,AREA_CLASSIFICATION,CITY_NAME,BEAT_NUMBER,...,RETURN_EGGS,REPLACED_EGGS,PROMO_EGGS,SINGLE_SKU_RATE,EGGS,RETAILER_CATEGORY,SALE_TYPE,RETAILER_ID,DISTRIBUTOR_ID,DISTRIBUTOR
0,F3122* Singla provison store,Cold,2022-10-27,1.0,135.0,135.0,None,Gurgaon-GT,Faridabad,3006.0,...,0.0,10.0,0.0,135.0,10.0,General Trader,Secondary_Sale,6003.0,4694.0,G2447* Anushka sales Agency
1,F3070* Amit store,Closed,2022-10-18,1.0,144.0,144.0,None,Gurgaon-GT,Faridabad,3006.0,...,0.0,20.0,0.0,144.0,10.0,General Trader,Secondary_Sale,5638.0,4694.0,G2447* Anushka sales Agency
2,F3003* AVNOOR ENTERPRISES,Closed,2022-10-21,4.0,436.0,436.0,None,Gurgaon-GT,Faridabad,5052.0,...,0.0,10.0,0.0,89.0,40.0,General Trader,Secondary_Sale,507.0,4694.0,G2447* Anushka sales Agency
3,F3020* The daily mart(Faridabad),Closed,2022-10-21,0.0,0.0,0.0,None,Gurgaon-GT,Faridabad,5050.0,...,0.0,12.0,0.0,0.0,0.0,General Trader,Secondary_Sale,812.0,4694.0,G2447* Anushka sales Agency
4,F3132* Daga Departmental store,Cold,2022-10-21,0.0,0.0,0.0,None,Gurgaon-GT,Faridabad,3006.0,...,0.0,12.0,0.0,0.0,0.0,General Trader,Secondary_Sale,6021.0,4694.0,G2447* Anushka sales Agency


In [36]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799805 entries, 0 to 799804
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   RETAILER_NAME           799805 non-null  object        
 1   ONBOARDING_STATUS       799805 non-null  object        
 2   DATE                    799804 non-null  datetime64[ns]
 3   QUANTITY                799805 non-null  float64       
 4   SKU_ORDER_PRICE_AMOUNT  799805 non-null  float64       
 5   NET_SALE                799805 non-null  float64       
 6   PARENT_RETAILER_NAME    339351 non-null  object        
 7   AREA_CLASSIFICATION     799805 non-null  object        
 8   CITY_NAME               799805 non-null  object        
 9   BEAT_NUMBER             799805 non-null  float64       
 10  BEAT_NUMBER_OPERATIONS  666506 non-null  float64       
 11  BRAND_TYPE              675406 non-null  object        
 12  ORDER_PRICE_AMOUNT      675406

In [22]:
len(result_df.columns)

29

In [5]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
output_excel_file = 'transformed_data_output.xlsx'  # Use the same file for all recipients

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')

# Open the Excel file in 'a' (append) mode using openpyxl engine
with pd.ExcelWriter(output_excel_file, engine='openpyxl', mode='a') as writer:
    def transform(query):
        # Execute Snowflake SQL Queries
        # Connecting to Snowflake
        conn = snowflake.connector.connect(
            user=username,
            password=password,
            account=account,
            warehouse=warehouse,
            database=database,
            schema=schema
        )
        cur = conn.cursor()
        cur.execute(f"USE {database}.{schema}")

        result_df = pd.read_sql_query(query, conn)

        # Identify columns of object type
        object_columns = result_df.select_dtypes(include='object').columns

        # Convert object columns to datetime
        for col in object_columns:
            try:
                converted_values = pd.to_datetime(result_df[col], errors='coerce')
                if converted_values.notna().any():
                    result_df[col] = converted_values
            except ValueError:
                # Handle any specific errors that might occur during conversion
                pass
        return result_df
    def send_email(name, email):
        # Create Excel attachment
        with open(output_excel_file, 'rb') as file:
            excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
            excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data_output.xlsx')

        # Create email message
        email_message = MIMEMultipart()
        email_message['From'] = sender_email
        email_message['To'] = email
        email_message['Subject'] = 'Transformed Data'
        email_message.attach(excel_attachment)

        # Customize email body
        email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
        email_message.attach(MIMEText(email_body, 'plain'))

        # Send email using SMTP
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, app_password)
            server.sendmail(sender_email, email, email_message.as_string())

    def fetch_transform_send():
        # Define queries for each sheet with specific columns
        queries = [
            """
            SELECT ID, NAME
            FROM eggozdb.maplemonk.my_sql_product_product
            """,
            """
            SELECT ID, CODE
            FROM eggozdb.maplemonk.my_sql_retailer_retailer
            """,
             """
            SELECT DATE, SKU_ORDER_PRICE_AMOUNT
            FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
            """
        ]

        # Iterate over queries and send emails
        for sheet_index, query in enumerate(queries, start=1):
            result_df = transform(query)

            # Save DataFrame to the current sheet
            excel_sheet_name = f'Sheet{sheet_index}'

            # Check if the sheet already exists
            if excel_sheet_name in writer.sheets:
                # If the sheet exists, you can choose to overwrite or use a different name
                excel_sheet_name += "_new"  # Use a different name (you can customize this logic)

            result_df.to_excel(writer, sheet_name=excel_sheet_name, index=False)

        # Send customized emails to each recipient for the current sheet
        for index, row in emails_df.iterrows():
            send_email(row['name'], row['email'])

        # Close the connection to Snowflake
        conn.close()

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)

BadZipFile: File is not a zip file

In [3]:
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\User\AppData\Local\Temp\ipykernel_12612\3879185800.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_12612\3879185800.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
C:\Users\User\AppData\Local\Temp\ipykernel_12612\3879185800.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)


KeyboardInterrupt: 

In [6]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Connecting to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
output_excel_file = 'transformed_data_output.xlsx'  # Use the same file for all recipients

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')
print(emails_df.columns)

# Open the Excel file in 'a' (append) mode using openpyxl engine
with pd.ExcelWriter(output_excel_file, engine='openpyxl', mode='a') as writer:

    def transform(query):
        # Execute Snowflake SQL Queries
        # Connecting to Snowflake
        conn = snowflake.connector.connect(
            user=username,
            password=password,
            account=account,
            warehouse=warehouse,
            database=database,
            schema=schema
        )
        cur = conn.cursor()
        cur.execute(f"USE {database}.{schema}")

        result_df = pd.read_sql_query(query, conn)

        # Identify date columns and convert them to date without timezone
        date_columns = [col for col in result_df.columns if result_df[col].dtype == 'datetime64[ns, UTC]']

        for date_column in date_columns:
            # Check if the date column exists before attempting to convert
            if date_column in result_df.columns:
                result_df[date_column] = result_df[date_column].dt.date

        return result_df

    def send_email(name, email):
        # Create Excel attachment
        with open(output_excel_file, 'rb') as file:
            excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
            excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data_output.xlsx')

        # Create email message
        email_message = MIMEMultipart()
        email_message['From'] = sender_email
        email_message['To'] = email
        email_message['Subject'] = 'Transformed Data'
        email_message.attach(excel_attachment)

        # Customize email body
        email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
        email_message.attach(MIMEText(email_body, 'plain'))

        # Send email using SMTP
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, app_password)
            server.sendmail(sender_email, email, email_message.as_string())

    def fetch_transform_send():
        # Define queries for each sheet with specific columns
        queries = [
            """
            SELECT QUANTITY, SKU_ORDER_PRICE_AMOUNT
            FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
            """,
            """
            SELECT ID, NAME
            FROM eggozdb.maplemonk.my_sql_product_product
            """,
            """
            SELECT ID, CODE
            FROM eggozdb.maplemonk.my_sql_retailer_retailer
            """
        ]

        # Iterate over queries and send emails
        for sheet_index, (query, sheet_name_suffix) in enumerate(zip(queries, ["Sheet1_akt", "Sheet2_abhi", "Sheet3_eggoz"]), start=1):
            result_df = transform(query)

            # Save DataFrame to the current sheet
            excel_sheet_name = f'{sheet_name_suffix}_new{sheet_index}'

            # Check if the sheet already exists
            if excel_sheet_name in writer.sheets:
                # If the sheet exists, you can choose to overwrite or use a different name
                excel_sheet_name += "_new"  # Use a different name (you can customize this logic)

            result_df.to_excel(writer, sheet_name=excel_sheet_name, index=False)

            # Send customized emails to each recipient for the current sheet
            for index, row in emails_df.iterrows():
                send_email(row['name'], row['email'])

        # Close the connection to Snowflake
        conn.close()

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)


Index(['name', 'email'], dtype='object')


BadZipFile: File is not a zip file

In [2]:
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\User\AppData\Local\Temp\ipykernel_10736\1275333277.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
